In [1]:
# 네이버 뉴스 크롤링
# https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=032&listType=paper&date=20240904&page=1
# 1 .기사의 href 링크제목과 링크를 크롤링
# 2. 수집한 링크를 순서대로 접근해서 상세내용 크롤링
# 3. 기사와 링크 내용 순으로 데이터를 결합
# 4. 결합한 데이터를 DB에 적재
# 5. 기사를 검색할수 있도록 streamlit를 구성해서 검색어로 제목을 검색하고 검색리스트를 보여준다음
# 사용자가 항목을 클릭하면 저장한 상세내역을 보여준다

In [2]:
import urllib.request
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [3]:
#경향신문 032, 국민 일보 005
op_news = '005'
url = f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid={op_news}&listType=paper&date=20240904'  
html = urllib.request.urlopen(url)
soup = BeautifulSoup(html,'html.parser')

In [4]:
def getMainArticle(result): # result는 list 
    articles = soup.select('#main_content > div.list_body.newsflash_body > ul')    
    for article in articles:
        result += [(data.text, data.attrs['href']) for data in article.find_all('a')[1:]]

In [5]:
import pandas as pd
result = []
getMainArticle(result)
result[:3]

[('이름 바꿔 점점 느는 정부 ‘쌈짓돈’ 특활비',
  'https://n.news.naver.com/mnews/article/005/0001722683'),
 ('[포토] 34년 만의 공휴일 국군의날… 기념식 예행연습',
  'https://n.news.naver.com/mnews/article/005/0001722676'),
 ('8월 물가 상승률 2.0%라는데… 장바구니 체감은 “글쎄”',
  'https://n.news.naver.com/mnews/article/005/0001722679')]

In [6]:
###########################

In [7]:
!pip install tqdm

In [8]:
from tqdm import tqdm
op_news = ['032','005','020','021','081','022','023','025','028','469']
result = []
for opn in tqdm(op_news):
    url = f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid={opn}&listType=paper&date=20240904'  
    html = urllib.request.urlopen(url)
    soup = BeautifulSoup(html,'html.parser')
    getMainArticle(result)
df1 = pd.DataFrame(result,columns=['title','link'])    

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.30it/s]


In [9]:
# 정규식
import re
url = 'https://n.news.naver.com/mnews/article/032/0003318882'
def getArticle(url):
    html = urllib.request.urlopen(url)
    soup = BeautifulSoup(html,'html.parser')
    article = soup.select('article')[0].text
    re_text = re.findall(r'[가-힣0-9\s.%]+',article)
    return ' '.join(re_text).replace('\n','').replace('\t','')
getArticle(url)

'대통령실  국민 안보의식 고취 오는 10월1일 건군 76주년 국군의날이 임시공휴일로 지정됐다.윤석열 대통령은 3일 국무회의를 통과한 국군의날 임시공휴일 지정안을 재가했다고 대통령실이 이날 밝혔다.대통령실은  엄중한 안보 상황 속에서 국방의 중요성과 국군의 존재 가치를 조명해 군의 사기를 진작시키고 국민의 안보 의식을 고취시키기 위해 임시공휴일로 지정했다 며  임시공휴일 지정에 따른 소비 진작 효과로 경제 활성화에도 이바지할 것으로 기대한다 고 밝혔다.국군의날은 1950년 10월1일 6 25전쟁에서 국군이 38선을 돌파한 날을 기념해 1956년 국가공휴일로 지정됐다. 1976년 법정 공휴일로 지정됐다가 1991년부터 공휴일에서 제외됐다. 국군의날이 공휴일이 된 것은 34년 만이다. 앞서 국민의힘은 지난달 25일 고위당정협의회에서 국군의날을 임시공휴일로 지정해달라고 정부에 요청했다.윤 대통령은 이날 국무회의를 통과한 전세사기특별법 개정안과 택시발전법 개정안  전통시장법 시행령 개정령안도 재가했다. 전세사기특별법은 전세사기 피해자에게 공공임대주택을 최대 20년간 지원하는 내용을 골자로 한다. 택시발전법은 이른바 택시 월급제의 전국 확대 시행을 2년 유예하는 내용이고 전통시장법 시행령에는 온누리상품권 사용처를 대폭 확대하는 내용이 담겼다.'

In [10]:
lists = [ getArticle(link) for link in tqdm(df1['link']) ]     

100%|████████████████████████████████████████████████████████████████████████████████| 203/203 [01:11<00:00,  2.84it/s]


In [11]:
df1['content'] = lists

In [12]:
df1.head()

,title,link,content
0,“나홀로 응급실 전담…못 받아준 환자가 더 걱정”,https://n.news.naver.com/mnews/article/032/000...,전공의 이탈 6개월째이대목동병원 가보니텅 빈 구급차 전용 주차장 정부가 추석연휴를 ...
1,인권위 ‘사정기관 인권침해’ 권리구제율 8%로 떨어졌다,https://n.news.naver.com/mnews/article/032/000...,2020년 19.6% 이후 하락세 뚜렷기각 각하율은 2년 연속 증가 일부 상임위원 ...
2,이재명·한동훈 공감 뒤 ‘지구당 부활’ 급물살···찬반 분분,https://n.news.naver.com/mnews/article/032/000...,한동훈 국민의힘 대표와 이재명 더불어민주당 대표가 1일 국회에서 열린 여야 대표 회...
3,여야 대표 ‘공감’에…힘 받는 ‘지구당 부활’,https://n.news.naver.com/mnews/article/032/000...,이르면 26일 합의 처리 전망풀뿌리 정치 부활 기대감 속양당 체제 고착화 등 우려도...
4,"내달 1일 국군의날, 임시공휴일로 지정",https://n.news.naver.com/mnews/article/032/000...,대통령실 국민 안보의식 고취 오는 10월1일 건군 76주년 국군의날이 임시공휴일로...


In [13]:
df1.shape

(203, 3)

In [14]:
df2 = df1.reset_index().rename(columns={'index':'id'})

In [15]:
# DB에 저장
from sqlalchemy import create_engine
# MySQL 데이터베이스 연결 생성
username = 'root'
password = '1234'
host = 'localhost'
database = 'mydatabase'
# SQLAlchemy 엔진 생성
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')
# DataFrame을 MySQL의 테이블로 삽입 (테이블이 없으면 생성)
df2.to_sql('news2', con=engine, if_exists='append', index=False)
# 데이터 삽입 후 연결 해제
engine.dispose()

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '0' for key 'news2.PRIMARY'")
[SQL: INSERT INTO news2 (id, title, link, content) VALUES (%(id)s, %(title)s, %(link)s, %(content)s)]
[parameters: [{'id': 0, 'title': '“나홀로 응급실 전담…못 받아준 환자가 더 걱정”', 'link': 'https://n.news.naver.com/mnews/article/032/0003318879', 'content': '전공의 이탈 6개월째이대목동병원 가보니텅 빈 구급차 전용 주차장 정부가 추석연휴를 앞두고 응급실 진료 중단을 막기 위해 군의관 등 인력을 보강하겠다고 밝힌 3일 서울 시내 한 대학병원 권역응급의료센터 앞 텅 빈 구급차 전용 주차장을 환자가 지나가고 있다. 정부는 응급 ... (1901 characters truncated) ...  이르는 상황까지는 아닌 것으로 판단하고 있다 고 했다. 박민수 복지부 2차관은 이날 브리핑에서  현재 응급환자는 점차 감소 추세에 있다 면서  인력 공백을 최소화하기 위해 군의관 공보의 파견  진료지원 간호사 촉탁의 채용 등을 통해 인력을 보강하고 있다 고 밝혔다.'}, {'id': 1, 'title': '인권위 ‘사정기관 인권침해’ 권리구제율 8%로 떨어졌다', 'link': 'https://n.news.naver.com/mnews/article/032/0003318880', 'content': '2020년 19.6% 이후 하락세 뚜렷기각 각하율은 2년 연속 증가 일부 상임위원 의무 등한시 탓 윤석열 정부가 출범한 2022년 이후 국가인권위원회가 권리구제를 한 비율은 줄고 기각 각하 등 종결한 비율은 증가한 것으로 나타났다.3일 윤종군 더불어민주당 의원실이 확 ... (700 characters truncated) ... 위원장의